#### Start the recognition server
Before doing anything we need to start the recognition server. In order to actually start processing the messages.

In [11]:
'''
import threading
import subprocess

def run_shell_script():
    subprocess.run(['bash', 'python ../recognition/detection/image-recognition/src/main.py'])

thread = threading.Thread(target=run_shell_script)
thread.start()

print("Shell script finished execution.")
'''

'\nimport threading\nimport subprocess\n\ndef run_shell_script():\n    subprocess.run([\'bash\', \'python ../recognition/detection/image-recognition/src/main.py\'])\n\nthread = threading.Thread(target=run_shell_script)\nthread.start()\n\nprint("Shell script finished execution.")\n'

### We first import the relevant libraries that we have installed with pip

In [12]:
import asyncio
import time
import cv2
import nats
import numpy as np
from nats.aio.msg import Msg
import nest_asyncio
import random
from asyncio import CancelledError

### We then import the project libraries that we will be using.
These are all from /common/ and /generated/ folders as they are the ones that are being shared in the project.

In [13]:
from generated.Inference_pb2 import InferenceList
from generated.Image_pb2 import ImageMessage
from generated.Inference_pb2 import Inference

We do "nest_asyncio.apply" in order to enable awaiting in the notebook environment. Otherwise, our nats server would not work.

In [14]:
nest_asyncio.apply()

#### Making a render util function
We make a function to draw a bounding box, given x,y coordinates [the Inference message] in order to easily visualize the detections. 

This is also partly done since, in the future, we will need to render text, distance... etc. This will cluster our main code function.

In [15]:
def render_detections(frame: np.ndarray, inference: Inference):
    for i in range(0, len(inference.bounding_box), 4):
        box = inference.bounding_box[i:i+4]
        x1, y1, x2, y2 = map(int, box)
        
        cv2.rectangle(
            frame,
            (x1, y1),
            (x2, y2),
            color=(0, 255, 0),
            thickness=2
        )
        
        # Add label with confidence
        label = f"{inference.class_name}: {inference.confidence:.2f}"
        cv2.putText(
            frame,
            label,
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (0, 255, 0),
            2
        )

#### Initiate the global variables

##### Video Capture and NATS connection
The nats connection basically connects to the server, and the video capture connects to the webcam.

In [16]:
cap = cv2.VideoCapture(0)
nt = await nats.connect("nats://localhost:4222")

##### Queue and Image ID map
We use a queue to store the incoming inference messages, and an image id map to store the images that we will be rendering. This is done due to the fact that the image is not available until the inference is processed <- which can take some time.

In [17]:
queue = asyncio.Queue()
image_id_map = {}

total_time_per_image = 0
total_images = 0
last_reset_time = 40

max_frame_age = 0.1
cur_frame_clean_time = 40

#### Making the on subscribe event function
This function is called whenever we receive a message from the nats server. We simply put the inference message in the queue.

In [18]:
async def on_message(msg: Msg):
    await queue.put(InferenceList.FromString(msg.data))

#### Subscribe to the NATS topic

In [19]:
await nt.subscribe("recognition/image_output", cb=on_message)

#### Main loop
This is the main loop of the program. It will read the webcam, send the image to the server, and render the detections.

In [20]:
try:
    while True:
        if cur_frame_clean_time > 30:
            cur_frame_clean_time = 0
            for image_id in image_id_map:
                if time.time() - image_id_map[image_id]["timestamp"] > max_frame_age:
                    image_id_map.pop(image_id)
            
        if abs(time.time() - last_reset_time) > 5:
            total_time_per_image = 0
            total_images = 0
            last_reset_time = time.time()

        total_images += 1
            
        ret, frame = cap.read()
        if not ret:
            continue

        _, compressed_image = cv2.imencode(".jpg", frame)
        
        image_id = random.randint(0, 1000000)

        msg = ImageMessage(
            image=compressed_image.tobytes(),
            camera_name="camera0",
            is_gray=False,
            image_id=image_id,
            height=frame.shape[0],
            width=frame.shape[1],
            timestamp=int(time.time() * 1000),
        )

        image_id_map[image_id] = {"frame": frame, "timestamp": time.time()}

        await nt.publish("recognition/image_input", msg.SerializeToString())
        await nt.flush()
        
        print("Sent image to server")

        if not queue.empty() and image_id in image_id_map:
            inference = await queue.get()
            for inference in inference.inferences:
                render_detections(image_id_map[image_id]["frame"], inference)
                
            cv2.imshow("frame", image_id_map[image_id]["frame"])
            cv2.waitKey(1)
            total_time_per_image += time.time() - image_id_map[image_id]["timestamp"]
            image_id_map.pop(image_id)
        
        time.sleep(total_time_per_image / total_images if total_images > 0 else 0.02)
except KeyboardInterrupt:
    print("Exiting...")
except CancelledError:
    print("Exiting...")
finally:
    cap.release()
    cv2.destroyAllWindows()

Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to server
Sent image to

nats: encountered error
Traceback (most recent call last):
  File "/Users/godbrigero/Documents/BLITZ/.venv/lib/python3.12/site-packages/nats/aio/client.py", line 2117, in _read_loop
    await self._ps.parse(b)
  File "/Users/godbrigero/Documents/BLITZ/.venv/lib/python3.12/site-packages/nats/protocol/parser.py", line 157, in parse
    await self.nc._process_pong()
  File "/Users/godbrigero/Documents/BLITZ/.venv/lib/python3.12/site-packages/nats/aio/client.py", line 1597, in _process_pong
    future.set_result(True)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/asyncio/futures.py", line 260, in set_result
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: CANCELLED: <Future cancelled>
